<h3>This code takes a database of all SWU cards and creates booster packs based on the official booster card distribution<h3>
<h5>You can use this to create card pools for playing Sealed or Draft with your friends online<h5>
<h5>Currently the bases and leaders are distributed uniformly, so you'll get rare and legendary bases and leaders more frequently than in real life <h5>

In [1]:
from datetime import datetime
today = datetime.today().strftime('%Y_%m_%d')
print(today)

2024_10_29


In [2]:

import random
import csv

collection = []
with open('../data/collection_detailed.csv', 'r', encoding='ISO-8859-1') as file:
    Reader = csv.reader(file)
    for line in Reader:
        collection.append(line)

full_card_pool = []

# This doesn't really do anything with this collection where all the cards have "count" of one
# but if you were using it with a csv of your own collection it would make sure each card is included for the number
# of times that the card actually exists in your collection 
for c in collection[1:]:
    count = int(c[4])
    card = c[:4] + c[5:]
    for i in range(count):
        full_card_pool.append(card)

# This creates one booster pack according to the FFG distribution
def get_booster(set='any'):
    card_pool = {}

    index = 0
    for c in full_card_pool:
        if set == 'any' or c[0] == set:
            card_pool[index] = c
            index += 1

    rare_leaders = {}
    leaders = {}
    bases = {}
    commons = {}
    uncommons = {}
    rares = {}
    legendaries = {}

    pack = []
    for k,v in card_pool.items():
        if v[4] == 'Leader' and v[17] in ['Rare','Special']: rare_leaders[k] = v
        elif v[4] == 'Leader': leaders[k] = v
        elif v[4] == 'Base' and v[17] != 'Rare': bases[k] = v
        elif v[17] == 'Common' : commons[k] = v
        elif v[17] == 'Uncommon' : uncommons[k] = v
        elif v[17] == 'Rare' : rares[k] = v
        elif v[17] == 'Legendary' : legendaries[k] = v
    print(len(leaders))
    print(len(rare_leaders))
    # CARD DISTRIBUTION
    # 1 Leader, 1 Base, 9 Commons, 3 Uncommons, 1 Rare/Legendary (Legendary 1 in 8 packs), 1 card of any type ()

    # 1 Leader
    random_number = random.randint(1, 6)
    print(random_number)
    if random_number == 6: leader = random.choice(list(rare_leaders.keys()))
    else: leader = random.choice(list(leaders.keys()))
    pack.append(card_pool[leader])
    
    # 1 Base
    base = random.choice(list(bases.keys()))
    pack.append(card_pool[base])
  
    # 9 Commons
    for i in range(9):
        common = random.choice(list(commons.keys()))
        pack.append(card_pool[common])

    # 3 Uncommons
    for i in range(3):
        uncommon = random.choice(list(uncommons.keys()))
        pack.append(card_pool[uncommon])

    # 1 Rare/Legendary (Legendary 1 in 8 packs)
    random_number = random.randint(1, 8)
    if len(legendaries) > 0 and random_number == 8: rare = random.choice(list(legendaries.keys()))
    else: rare = random.choice(list(rares.keys()))
    pack.append(card_pool[rare])

    # 1 card of any type
    any_card = random.choice(list(card_pool.keys()))
    any_card_type = card_pool[any_card][4]
    while any_card_type == 'Leader' or any_card_type == 'Base':
        any_card = random.choice(list(card_pool.keys()))
        any_card_type = card_pool[any_card][4]
    pack.append(card_pool[any_card])

    return(pack)





# USE get_boosters if you want to create multiple boosters from the same collection
# especially if you want to build them from your real life collection

def get_boosters(count, set='any'):
    card_pool = {}
    packs = []
    
    # If you set remove_from_pool to True it will remove cards from the card pool as it adds them to boosters.
    # I use this when creating boosters from my personal card collection so it only uses cards I actually own.

    remove_from_pool = True

    index = 0
    for c in full_card_pool:
        if set == 'any' or c[0] == set:
            card_pool[index] = c
            index += 1


    for i in range(count):
        leaders = {}
        rare_leaders = {}
        bases = {}
        commons = {}
        uncommons = {}
        rares = {}
        legendaries = {}

        pack = []
        for k,v in card_pool.items():
            if v[4] == 'Leader' and v[17] in ['Rare','Special']: rare_leaders[k] = v
            elif v[4] == 'Leader': leaders[k] = v
            
            elif v[4] == 'Base' and v[17] != 'Rare': bases[k] = v
            elif v[17] == 'Common' : commons[k] = v
            elif v[17] == 'Uncommon' : uncommons[k] = v
            elif v[17] == 'Rare' : rares[k] = v
            elif v[17] == 'Legendary' : legendaries[k] = v

        # CARD DISTRIBUTION
        # 1 Leader, 1 Base, 9 Commons, 3 Uncommons, 1 Rare/Legendary (Legendary 1 in 8 packs), 1 card of any type ()

        # 1 Leader with 1 in 6 chance of rare (I included Specials with rares)
        random_number = random.randint(1, 6)
        if random_number == 6: leader = random.choice(list(rare_leaders.keys()))
        else: leader = random.choice(list(leaders.keys()))
        pack.append(card_pool[leader])
        if remove_from_pool: del card_pool[leader]
        # 1 Base
        base = random.choice(list(bases.keys()))
        pack.append(card_pool[base])
        if remove_from_pool: del card_pool[base]
        # 9 Commons
        for i in range(9):
            common = random.choice(list(commons.keys()))
            pack.append(card_pool[common])
            if remove_from_pool: del card_pool[common]
            if remove_from_pool: del commons[common]
        # 3 Uncommons
        for i in range(3):
            uncommon = random.choice(list(uncommons.keys()))
            pack.append(card_pool[uncommon])
            if remove_from_pool: del card_pool[uncommon]
            if remove_from_pool: del uncommons[uncommon]
        # 1 Rare/Legendary (Legendary 1 in 8 packs)
        random_number = random.randint(1, 8)
        if len(legendaries) > 0 and random_number == 8: rare = random.choice(list(legendaries.keys()))
        else: rare = random.choice(list(rares.keys()))
        pack.append(card_pool[rare])
        if remove_from_pool: del card_pool[rare]
        # 1 card of any type
        any_card = random.choice(list(card_pool.keys()))
        any_card_type = card_pool[any_card][4]
        while any_card_type == 'Leader' or any_card_type == 'Base':
            any_card = random.choice(list(card_pool.keys()))
            any_card_type = card_pool[any_card][4]
        pack.append(card_pool[any_card])
        if remove_from_pool: del card_pool[any_card]

        packs.append(pack)
    return(packs)






<p>get_booster(set)<p> 
<p>where set is the three letter code for the set (lower case)<p>
<p>if you leave the set blank, it will create a booster with cards from all sets <p>

In [3]:
sor_booster = get_booster('sor')
shd_booster = get_booster('shd')
all_cards_booster = get_booster()

11
8
4
10
8
2
21
16
6


<p>get_boosters(n, set)<p> 
<p>where n is the number of packs you want to create, and set is the three letter code for the set (lower case)<p>
<p>if you leave the set blank, it will create boosters with cards from all sets <p>

In [4]:
sor_boosters = get_boosters(2,'sor')
shd_boosters = get_boosters(2,'shd')
all_cards_boosters = get_boosters(2)

In [5]:
sor_booster = get_booster('sor')


11
8
1


<h3>This code will give you images of the cards for a random booster<h3>

In [6]:
#Change swu set to 'sor', 'shd', or 'any' for cards from all sets
swu_set = 'shd'




# DON'T CHANGE THIS CODE
booster = get_booster(swu_set)
from IPython.display import display, HTML

html = "<div style='display: flex; flex-wrap: wrap;'>"
for i, lst in enumerate(booster):
    url = lst[16] 
    html += f"""
    <div style='margin: 10px;'>
        <img src="{url}" alt="Image {i+1}" style="width: 250px; height: auto;">
    </div>
    """
    if (i + 1) % 4 == 0:
        html += "<br>"

html += "</div>"

# Display the HTML
display(HTML(html))

10
8
4


<h3> This code will print out the names and card numbers for a random booster pack<h3>

In [7]:
#Change swu set to 'sor', 'shd', or 'any' for cards from all sets
swu_set = 'sor'

#Leave this code alone
booster = get_booster(swu_set)
for i in booster:
    if i[3]: print(f"{i[0]} {i[1]} -- {i[2]}: {i[3]} -- {i[17]}")
    else: print(f"{i[0]} {i[1]} -- {i[2]} -- {i[17]}")

11
8
5
sor 015 -- Boba Fett: Collecting the Bounty -- Common
sor 029 -- Administrator's Tower: Cloud City -- Common
sor 032 -- Scout Bike Pursuer -- Common
sor 195 -- Auzituck Liberator Gunship -- Common
sor 220 -- Surprise Strike -- Common
sor 232 -- AT-ST -- Common
sor 195 -- Auzituck Liberator Gunship -- Common
sor 169 -- Keep Fighting -- Common
sor 130 -- First Legion Snowtrooper -- Common
sor 208 -- Outer Rim Headhunter -- Common
sor 063 -- Cloud City Wing Guard -- Common
sor 131 -- Fifth Brother: Fear Hunter -- Uncommon
sor 191 -- Vanguard Ace -- Uncommon
sor 131 -- Fifth Brother: Fear Hunter -- Uncommon
sor 091 -- The Emperor's Legion -- Rare
sor 231 -- TIE Advanced -- Uncommon


In [8]:
aspects = {
        "['Aggression', 'Villainy']":"Red Black",
        "['Command', 'Villainy']":"Green Black",
        "['Vigilance', 'Villainy']": "Blue Black",
        "['Cunning', 'Villainy']": "Yellow Black",
        "['Aggression', 'Heroism']":"Red White",
        "['Command', 'Heroism']":"Green White",
        "['Vigilance', 'Heroism']": "Blue White",
        "['Cunning', 'Heroism']": "Yellow White",
        "['Heroism']": "White",
        "['Villainy']": "Black",
        "": "None",
        "['Aggression']":"Red",
        "['Command']":"Green",
        "['Vigilance']": "Blue",
        "['Cunning']": "Yellow",
        "['Aggression', 'Aggression']":"Red Red",
        "['Command', 'Command']":"Green Green",
        "['Vigilance', 'Vigilance']": "Blue Blue",
        "['Cunning', 'Cunning']": "Yellow Yellow",
           }

In [11]:
nick_boosters = get_boosters(6,'sor')
josh_boosters = get_boosters(6,'shd')


pool = []
leaders = []
bases = []
cards = []

for pack in nick_boosters:
    for card in pack:
        if card[4] == 'Leader': leaders.append(card)
        elif card[4] == 'Base': bases.append(card)
        else: cards.append(card)

sorted_cards = sorted(cards, key=lambda x:(x[5],x[8]))

for i in leaders: pool.append(i)
for i in bases: pool.append(i)
for i in sorted_cards: pool.append(i)

card_text = []
for i in pool:
    card_text.append(f"{i[2]} - {aspects[i[5]]} - {i[8]}")

print(len(card_text))
for i in card_text:print(i)

with open(f"../data/sealed_sets/Nick_Sealed_Cards_{today}.txt", "w") as file:
    for i in card_text:
        file.write(f"{i}\n")

pool = []
leaders = []
bases = []
cards = []

for pack in josh_boosters:
    for card in pack:
        if card[4] == 'Leader': leaders.append(card)
        elif card[4] == 'Base': bases.append(card)
        else: cards.append(card)

sorted_cards = sorted(cards, key=lambda x:(x[5],x[8]))


for i in sorted_cards: pool.append(i)

card_text = []
for i in pool:
    card_text.append(f"{i[2]} - {aspects[i[5]]} - {i[8]}")

print(len(card_text))
for i in card_text:print(i)

with open(f"../data/sealed_sets/Josh_Sealed_Cards_{today}.txt", "w") as file:
    for i in card_text:
        file.write(f"{i}\n")



96
Boba Fett - Yellow Black - 5
Leia Organa - Green White - 5
Chewbacca - Blue White - 7
Iden Versio - Blue Black - 6
IG-88 - Red Black - 5
Boba Fett - Yellow Black - 5
Chopper Base - Yellow - 
Administrator's Tower - Yellow - 
Command Center - Green - 
Catacombs of Cadera - Red - 
Jedha City - Yellow - 
Kestro City - Red - 
Catacombs of Cadera - Red - 
Underworld Thug - None - 2
Corellian Freighter - None - 5
Corellian Freighter - None - 5
SpecForce Soldier - Red White - 1
Fighters For Freedom - Red White - 3
Fighters For Freedom - Red White - 3
Force Choke - Red Black - 2
Ruthless Raider - Red Black - 6
Emperor Palpatine - Red Black - 8
Cantina Braggart - Red - 1
Force Throw - Red - 1
Jedha Agitator - Red - 2
Open Fire - Red - 3
Open Fire - Red - 3
Ardent Sympathizer - Red - 3
Disabling Fang Fighter - Red - 3
Attack Pattern Delta - Green Green - 3
Alliance Dispatcher - Green White - 1
Alliance Dispatcher - Green White - 1
Rebel Assault - Green White - 1
Admiral Ackbar - Green White -

In [ ]:
with open('../data/sealed_sets/Nick_Sealed_cards_2024_10_29.txt)